In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import statsmodels.api as sm
import warnings
import shap
from sklearn.model_selection import StratifiedKFold
from pycaret.regression import *
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import ExtraTreeRegressor
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import KNNImputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostClassifier, Pool
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.metrics import mean_squared_error
from pycaret.regression import *
from sklearn.impute import KNNImputer

In [3]:
warnings.simplefilter(action='ignore')
pd.set_option('display.max_columns', None)

# Data

In [4]:
train_data = pd.read_csv("./train.csv")
train_data

PRODUCT_ID  Y_Class  Y_Quality         TIMESTAMP     LINE PRODUCT_CODE  \
0    TRAIN_000        1   0.533433   2022-06-13 5:14  T050304         A_31   
1    TRAIN_001        2   0.541819   2022-06-13 5:22  T050307         A_31   
2    TRAIN_002        1   0.531267   2022-06-13 5:30  T050304         A_31   
3    TRAIN_003        2   0.537325   2022-06-13 5:39  T050307         A_31   
4    TRAIN_004        1   0.531590   2022-06-13 5:47  T050304         A_31   
..         ...      ...        ...               ...      ...          ...   
593  TRAIN_593        1   0.526546  2022-09-08 14:30  T100306         T_31   
594  TRAIN_594        0   0.524022  2022-09-08 22:38  T050304         A_31   
595  TRAIN_595        0   0.521289  2022-09-08 22:47  T050304         A_31   
596  TRAIN_596        1   0.531375  2022-09-08 14:38  T100304         O_31   
597  TRAIN_597        1   0.533702  2022-09-08 14:46  T100306         O_31   

      X_1   X_2  X_3   X_4   X_5  X_6   X_7   X_8   X_9  X_10   X_11   X_12  \
0     NaN   NaN  NaN   NaN   NaN  NaN   NaN   NaN   NaN   NaN    NaN    NaN   
1     NaN   NaN  NaN   NaN   NaN  NaN   NaN   NaN   NaN   NaN    NaN    NaN   
2     NaN   NaN  NaN   NaN   NaN  NaN   NaN   NaN   NaN   NaN    NaN    NaN   
3     NaN   NaN  NaN   NaN   NaN  NaN   NaN   NaN   NaN   NaN    NaN    NaN   
4     NaN   NaN  NaN   NaN   NaN  NaN   NaN   NaN   NaN   NaN    NaN    NaN   
..    ...   ...  ...   ...   ...  ...   ...   ...   ...   ...    ...    ...   
593   2.0  95.0  0.0  45.0  10.0  0.0  50.0  10.0  52.0   2.0  468.9  473.9   
594   NaN   NaN  NaN   NaN   NaN  NaN   NaN   NaN   NaN   NaN    NaN    NaN   
595   NaN   NaN  NaN   NaN   NaN  NaN   NaN   NaN   NaN   NaN    NaN    NaN   
596  40.0  94.0  0.0  45.0  11.0  0.0  45.0  10.0  31.0   2.0  505.8  510.9   
597  21.0  87.0  0.0  45.0  10.0  0.0  61.0  10.0  52.0   2.0  470.6  474.9   

      X_13  X_14  X_15   X_16   X_17   X_18   X_19   X_20   X_21   X_22  X_23  \
0      NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   NaN   
1      NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   NaN   
2      NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   NaN   
3      NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   NaN   
4      NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   NaN   
..     ...   ...   ...    ...    ...    ...    ...    ...    ...    ...   ...   
593  459.2   0.0   0.0  247.5  248.0  247.3  250.0  527.0  538.0  522.0  0.19   
594    NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   NaN   
595    NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   NaN   
596  497.1   0.0   0.0  247.5  248.1  247.3  250.0  489.0  498.0  485.0  0.19   
597  462.9   0.0   0.0  247.5  248.0  247.3  250.0  525.0  534.0  520.0  0.19   

     X_24  X_25   X_26   X_27   X_28   X_29   X_30   X_31   X_32   X_33  \
0     NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
1     NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2     NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3     NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4     NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
..    ...   ...    ...    ...    ...    ...    ...    ...    ...    ...   
593  0.19  0.19  155.0  155.0  155.0  155.0  185.0  185.0  185.0  185.0   
594   NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
595   NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
596  0.19  0.19  155.0  155.0  155.0  155.0  185.0  185.0  185.0  185.0   
597  0.20  0.19  155.0  155.0  155.0  155.0  185.0  185.0  185.0  185.0   

      X_34   X_35   X_36   X_37   X_38  X_39  X_40  X_41  X_42  X_43  X_44  \
0      NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1      NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2      NaN

In [ ]:
colls = []
for coll in train_data :
    if 'X' in coll and train_data[coll].isna().sum() == 598 :
        colls.append(coll)
print(len(colls))

In [ ]:
train_data.drop(colls, inplace = True, axis = 1)

In [ ]:
train_data.loc[train_data.PRODUCT_CODE == 'A_31','PRODUCT_CODE'] = 1
train_data.loc[train_data.PRODUCT_CODE == 'T_31','PRODUCT_CODE'] = 2
train_data.loc[train_data.PRODUCT_CODE == 'O_31','PRODUCT_CODE'] = 3

train_data.drop(['PRODUCT_ID', 'TIMESTAMP'], axis = 1, inplace = True)
train_data

In [ ]:
T05 = train_data[(train_data['LINE'] == 'T050304')|(train_data['LINE'] == 'T050307')]
T01 = train_data[(train_data['LINE'] == 'T010305')|(train_data['LINE'] == 'T010306')]
T10 = train_data[(train_data['LINE'] == 'T100304')|(train_data['LINE'] == 'T100306')]

In [5]:
#44
test_data = pd.read_csv("./test.csv")
test_data

PRODUCT_ID         TIMESTAMP     LINE PRODUCT_CODE  X_1   X_2  X_3   X_4  \
0     TEST_000   2022-09-09 2:01  T100306         T_31  2.0  94.0  0.0  45.0   
1     TEST_001   2022-09-09 2:09  T100304         T_31  2.0  93.0  0.0  45.0   
2     TEST_002   2022-09-09 8:42  T100304         T_31  2.0  95.0  0.0  45.0   
3     TEST_003  2022-09-09 10:56  T010305         A_31  NaN   NaN  NaN   NaN   
4     TEST_004  2022-09-09 11:04  T010306         A_31  NaN   NaN  NaN   NaN   
..         ...               ...      ...          ...  ...   ...  ...   ...   
305   TEST_305  2022-11-05 11:18  T100306         T_31  2.0  91.0  0.0  45.0   
306   TEST_306  2022-11-05 16:39  T100304         T_31  2.0  96.0  0.0  45.0   
307   TEST_307  2022-11-05 16:47  T100306         T_31  2.0  91.0  0.0  45.0   
308   TEST_308  2022-11-05 20:53  T100306         T_31  2.0  95.0  0.0  45.0   
309   TEST_309  2022-11-05 21:01  T100306         T_31  2.0  87.0  0.0  45.0   

      X_5  X_6   X_7   X_8   X_9  X_10   X_11   X_12   X_13  X_14  X_15  \
0    10.0  0.0  51.0  10.0  52.0   2.0  469.6  474.4  461.0   0.0   0.0   
1    11.0  0.0  45.0  10.0  31.0   2.0  506.6  511.1  500.1   0.0   0.0   
2    11.0  0.0  45.0  10.0  31.0   2.0  506.6  511.4  500.6   0.0   0.0   
3     NaN  NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
4     NaN  NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
..    ...  ...   ...   ...   ...   ...    ...    ...    ...   ...   ...   
305  10.0  0.0  51.0  10.0  52.0   2.0  502.5  511.9  488.3   0.0   0.0   
306  11.0  0.0  45.0  10.0  31.0   2.0  513.7  518.9  505.0   0.0   0.0   
307  10.0  0.0  50.0  10.0  52.0   2.0  502.8  511.6  489.8   0.0   0.0   
308  10.0  0.0  51.0  10.0  52.0   2.0  503.2  512.6  489.8   0.0   0.0   
309  10.0  0.0  51.0  10.0  52.0   2.0  503.3  512.1  489.5   0.0   0.0   

      X_16   X_17   X_18   X_19   X_20   X_21   X_22  X_23  X_24  X_25   X_26  \
0    247.5  248.0  247.2  250.0  526.0  537.0  521.0  0.19  0.19  0.19  155.0   
1    247.6  248.2  247.4  250.0  489.0  496.0  484.0  0.19  0.19  0.19  155.0   
2    247.6  247.9  247.4  250.0  488.0  496.0  484.0  0.19  0.19  0.19  155.0   
3      NaN    NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN    NaN   
4      NaN    NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN    NaN   
..     ...    ...    ...    ...    ...    ...    ...   ...   ...   ...    ...   
305  247.5  248.1  247.3  250.0  492.0  506.0  483.0  0.19  0.20  0.19  155.0   
306  247.5  248.1  247.4  250.0  481.0  490.0  477.0  0.19  0.19  0.19  155.0   
307  247.6  248.0  247.3  250.0  491.0  506.0  483.0  0.19  0.20  0.19  155.0   
308  247.5  248.0  247.3  250.0  491.0  505.0  483.0  0.19  0.20  0.19  155.0   
309  247.5  248.0  247.3  250.0  491.0  505.0  483.0  0.19  0.20  0.19  155.0   

      X_27   X_28   X_29   X_30   X_31   X_32   X_33   X_34   X_35   X_36  \
0    155.0  155.0  155.0  185.0  185.0  185.0  185.0  155.0  155.0  155.0   
1    155.0  155.0  155.0  185.0  185.0  185.0  185.0  155.0  155.0  155.0   
2    155.0  155.0  155.0  185.0  185.0  185.0  185.0  155.0  155.0  155.0   
3      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
..     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
305  155.0  155.0  155.0  185.0  185.0  185.0  185.0  155.0  155.0  155.0   
306  155.0  155.0  155.0  185.0  185.0  185.0  185.0  155.0  155.0  155.0   
307  155.0  155.0  155.0  185.0  185.0  185.0  185.0  155.0  155.0  155.0   
308  155.0  155.0  155.0  185.0  185.0  185.0  185.0  155.0  155.0  155.0   
309  155.0  155.0  155.0  185.0  185.0  185.0  185.0  155.0  155.0  155.0   

      X_37   X_38  X_39  X_40  X_41  X_42  X_43  X_44  X_45  X_46  X_47  X_48  \
0    155.0  0.075   2.0  67.2  68.9  67.5  79.8  70.9  80.5  71.2  67.0  76.4   
1    155.0  0.079   2.0  63.7  64.6  64.7  78.6  69.0  82.2  70.4  64.7  74.3   
2